In [16]:
import os
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
read_lexicon()
op_set = positive_lexicon + negative_lexicon

In [39]:
import pandas as pd
import numpy as np

def calculate(category, tipe):
    df = pd.read_csv('Results/Sentiwordnet/hasil_'+ tipe + category +'.csv')
    tp, tn, fp, fn = 0, 0, 0, 0
    p = 'positive'
    n = 'negative'
    for index in range(0, len(df)):
        label = df['label'][index]
        predict = df['predict'][index]
        
        if label == p and predict == p:
            tp += 1
        elif label == n and predict == n:
            tn += 1
        elif label == n and predict == p:
            fp += 1
        elif label == p and predict == n:
            fn += 1
    
    precision = round(tp / (tp + fp), 2)
    recall = round(tp / (tp + fn), 2)
    f1 = round(2 * ((precision * recall) / (precision + recall)), 2)
    accuracy = round((tp+tn)/(tp+tn+fp+fn), 2)
    p = tp / (tp + fp)
    r = tp / (tp + fn)
    f = 2 * ((p * r) / (p + r))
    
    
    print(tp, '\t', tn, '\t', fp, '\t', fn, '\t', '\t', precision, '\t', recall, '\t', f1, '\t', '\t', accuracy)
    return tp, tn, fp, fn, p, r, f, accuracy

In [40]:
def preprocessing(sentence):
    return sentence.lower()

def calculate_f1(p, r):
    return round(2*((p*r)/(p+r)),2)

negation = [
    "afraid",
    "can't",
    "cannot",
    "deny",
    "mean",
    "negate",
    "negation",
    "negative",
    "neither",
    "never",
    "no",
    "non",
    "none",
    "nor",
    "not",
    "nothing",
    "refusal",
    "refuse",
    "reject",
    "rejection",
#     "don't",
#     "shouldn't",
#     "wouldn't",
#     "didn't",
#     "though",
#     "although",
#     "wasn't",
#     "isn't",
#     "but"
]

In [41]:
import numpy as np

def print_hasil(tipe):
    sf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'precision', 'recall', 'f1', 'accuracy'])
    print('tp', '\t', 'tn', '\t', 'fp', '\t', 'fn', '\t', '\t', 'prec', '\t', 'rec', '\t', 'f1', '\t', '\t', 'accuracy')
    precision = []
    recall = []
    f1 = []
    cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'PRICES']
    for c in cat:
        a, b, c, d, e, f, g, h = calculate(c, tipe)
        precision.append(e)
        recall.append(f)
        f1.append(g)
        sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g, 'accuracy': h}, ignore_index=True)
    sf.to_excel("Results/Calculation/sentiment-"+ tipe +".xlsx")
    print('', '\t', '', '\t', '', '\t', '', '\t', '\t', round(np.mean(precision),2), '\t', round(np.mean(recall),2), '\t', calculate_f1(round(np.mean(precision),2), round(np.mean(recall),2)), '\t', '\t')

In [42]:
import string

def predict_lexicon(opinions, positive_lexicon, negative_lexicon, sent):
    endscore = 0
    for opinion in opinions:
        if opinion in positive_lexicon:
            endscore += 1
        elif opinion in negative_lexicon:
            endscore -= 1
                
        
        words = sent.split(' ')
        word_around = []
        for x in range(0, len(words)):
            if words[x] in string.punctuation:
                continue
            try:
                if (words[x+1] == opinion) or (words[x+2] == opinion) or (words[x+3] == opinion) or (words[x+4] == opinion):
                    word_around.append(words[x])
                elif (words[x-1] == opinion):
                    word_around.append(words[x])
            except:
                pass
        for neg in negation:
            if neg in word_around:
                endscore *= (-1)
                break

    if endscore > 0:
        polarity = 'positive'
    else:
        polarity = 'negative'
           
    return polarity

In [43]:
import pandas as pd
def run(category):
    df = pd.read_csv('Results/'+ category +'.csv')
    sf = pd.DataFrame(columns = ['id','review', 'opinion', 'label', 'predict'])
    
    of = pd.read_csv('Results/Opinion/' + category + '.csv')
    positive_lexicon = []
    negative_lexicon = []
    for i in range(0, len(of)):
        if of['sentimen'][i] == 'positive':
            positive_lexicon.append(of['opini'][i])
        else:
            negative_lexicon.append(of['opini'][i])
            
    print(len(df))
    count = 0
    for index in range(0, len(df)):
        opinion = []
        if type(df['opinion'][index]) != float:
            opinion = df['opinion'][index].split('|')
            
        
            
        prediction = predict_lexicon(opinion, positive_lexicon, negative_lexicon, preprocessing(df['review'][index]))
        
        sf = sf.append({'id': df['id'][index], 
            'review': df['review'][index],
            'opinion': df['opinion'][index],
            'label': df['label'][index],
            'predict': prediction
           }, ignore_index=True)
        count += 1
    print(count)
    sf.to_csv("Results/Sentiwordnet/hasil_cir_"+ category +".csv")
    sf.to_excel("Results/Sentiwordnet/hasil_cir_"+ category +".xlsx")

In [44]:
run('AMBIENCE')
run('FOOD')
run('SERVICE')
run('PRICES')

107
107
177
177
122
122
80
80


In [13]:
print_hasil('cir_')

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
77 	 10 	 8 	 16 	 	 0.91 	 0.83 	 0.87 	 	 0.78
120 	 25 	 11 	 21 	 	 0.92 	 0.85 	 0.88 	 	 0.82
73 	 27 	 11 	 9 	 	 0.87 	 0.89 	 0.88 	 	 0.83
38 	 32 	 6 	 6 	 	 0.86 	 0.86 	 0.86 	 	 0.85
 	  	  	  	 	 0.89 	 0.86 	 0.87 	 	


In [48]:
print_hasil('cir_')

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
72 	 10 	 8 	 17 	 	 0.9 	 0.81 	 0.85 	 	 0.77
118 	 25 	 11 	 23 	 	 0.91 	 0.84 	 0.87 	 	 0.81
73 	 29 	 11 	 9 	 	 0.87 	 0.89 	 0.88 	 	 0.84
32 	 34 	 3 	 11 	 	 0.91 	 0.74 	 0.82 	 	 0.82
 	  	  	  	 	 0.9 	 0.82 	 0.86 	 	


In [36]:
print_hasil('cir_')

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
78 	 10 	 7 	 15 	 	 0.92 	 0.84 	 0.88 	 	 0.8
119 	 25 	 11 	 22 	 	 0.92 	 0.84 	 0.88 	 	 0.81
73 	 28 	 11 	 9 	 	 0.87 	 0.89 	 0.88 	 	 0.83
32 	 34 	 3 	 11 	 	 0.91 	 0.74 	 0.82 	 	 0.82
 	  	  	  	 	 0.9 	 0.83 	 0.86 	 	
